# Задача: Получить актульный список действующих услуг и сервисов, которые находятся в каталоге услуг mos.ru для последующих задач.

Для ускорения парсинга будем пользоваться несколькими потоками, так как есть задержка при открытии страницы.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from tqdm import tqdm
import threading

In [2]:
#читаем файл с ссылками на каталог
sept = pd.read_excel('Ссылки.xlsx',dtype='str')

In [3]:
#дополняем каталог ссылками с раскрывающимся списком
new_df = sept.copy()
for i in tqdm(range(len(sept))):
    url = sept.iloc[i]['ссылка']
    browser = webdriver.Chrome()

    browser.get(url)
    generated_html = browser.page_source
    browser.quit()
    generated_html
    soup = BeautifulSoup(generated_html, 'html.parser')
    product_names = soup.find_all(class_='mos-catalog-service-group__link')
    product_names += soup.find_all(class_='mos-catalog_subservice-title')
    for i in range(len(product_names)):
        link_dop = str(product_names[i]).split('href="')[1].split('"')[0]
        link_dop = "https://www.mos.ru" + link_dop
        new_df = pd.concat([new_df, pd.DataFrame(data={"ссылка": [link_dop]})], ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [19:23<00:00,  9.78s/it]


In [4]:
new_df.to_excel('ссылки_для_ГУ.xlsx')

In [5]:
new_df = pd.read_excel('ссылки_для_ГУ.xlsx')

In [6]:
url_gu = pd.DataFrame(columns=["Ссылка", "Наименование_в_каталоге"])

In [7]:
# функция для извлечения ссылки на услуги и ее наименования
def function_1(j,sem):
    url = new_df.iloc[j]['ссылка']
    browser = webdriver.Chrome()

    browser.get(url)
    generated_html = browser.page_source
    browser.quit()
    generated_html
    soup = BeautifulSoup(generated_html, 'html.parser')
    product_names = soup.find_all(class_='mos-service-item')
    for i in range(len(product_names)):
        global url_gu
        link_dop = '-'
        name_gu = '-'
        link_dop = str(product_names[i]).split('href="')[1].split('"')[0]
        if 'aria-label="' in str(product_names[i]):
            name_gu = str(product_names[i]).split('aria-label="')[1].split('"')[0]
        if "aria-label='" in str(product_names[i]):
            name_gu = str(product_names[i]).split("aria-label='")[1].split("'")[0]
        url_gu = pd.concat([url_gu, pd.DataFrame(data={"Ссылка": [link_dop], "Наименование_в_каталоге": [name_gu]})], ignore_index=True)
    sem.release()

In [8]:
# function_1 ограничим максимальное количество потоков до 6

def main():
    sem = threading.Semaphore(6)
    for j in tqdm(new_df.index):
        sem.acquire()
        threading.Thread(target=function_1, args=(j, sem)).start()

if __name__ ==  "__main__":
    main()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 323/323 [11:20<00:00,  2.11s/it]


In [9]:
url_gu

,Ссылка,Наименование_в_каталоге
0,https://www.mos.ru/pgu2/landing/340585/,Услуга Посмотреть информацию о посещении и пит...
1,https://www.mos.ru/pgu2/landing/target/7700000...,Услуга Запись (перевод) в детский сад
2,https://www.mos.ru/pgu2/landing/target/7700000...,Услуга Компенсация платы за детский сад
3,https://www.mos.ru/pgu2/landing/target/7700000...,"Услуга Запись в кружки, спортивные секции, дом..."
4,https://pay.mos.ru/mospaynew/newportal/,Услуга Оплата кружков и услуг образования
...,...,...
3966,https://www.mos.ru/pgu2/landing/target/7700000...,Услуга Переоформление лицензии на оборот нарко...
3967,https://www.mos.ru/pgu2/landing/target/7700000...,Услуга Переоформление лицензии на оборот нарко...
3968,https://www.mos.ru/pgu2/landing/target/7700000...,Услуга Переоформление лицензии на оборот нарко...
3969,https://www.mos.ru/pgu2/landing/target/7700000...,Услуга Прекращение действия лицензии на оборот...


In [10]:
# удаляем дубликаты
url_gu_1 = url_gu.copy()
url_gu_1 = url_gu_1.drop_duplicates(ignore_index=True)

In [11]:
# удалить "Услуга " в столбце "Наименование_в_каталоге"
for i in range(len(url_gu_1)):
    if url_gu_1.loc[i,'Наименование_в_каталоге'][:7] == 'Услуга ':
        url_gu_1.loc[i]['Наименование_в_каталоге'] = url_gu_1.loc[i,'Наименование_в_каталоге'][7:]
        url_gu_1.loc[i]['Ссылка'] = url_gu_1.loc[i,'Ссылка'][18:]

In [12]:
url_gu_1.to_excel('ссылки_на_ГУ_из_справочника.xlsx')

In [13]:
url_gu_1 = pd.read_excel('ссылки_на_ГУ_из_справочника.xlsx')
url_gu_1.drop('Unnamed: 0', axis= 1 , inplace= True )
for i in range(len(url_gu_1)):
    if url_gu_1.loc[i]['Ссылка'][0] != '/':
        url_gu_1.loc[i, 'Ссылка'] = '/'+str(url_gu_1.loc[i, 'Ссылка'])
url_gu_1['Ссылка'] = url_gu_1['Ссылка'].apply(lambda x: "https://www.mos.ru" + str(x))
url_gu_1

,Ссылка,Наименование_в_каталоге
0,https://www.mos.ru/pgu2/landing/340585/,Посмотреть информацию о посещении и питании в ...
1,https://www.mos.ru/pgu2/landing/target/7700000...,Запись (перевод) в детский сад
2,https://www.mos.ru/pgu2/landing/target/7700000...,Компенсация платы за детский сад
3,https://www.mos.ru/pgu2/landing/target/7700000...,"Запись в кружки, спортивные секции, дома творч..."
4,https://www.mos.ru/mospaynew/newportal/,Оплата кружков и услуг образования
...,...,...
690,https://www.mos.ru/pgu2/landing/target/7700000...,Согласие на субаренду либо переуступку прав ар...
691,https://www.mos.ru/pgu2/landing/target/7700000...,Выкуп арендуемой недвижимости у города субъект...
692,https://www.mos.ru/ggi/function/opredelenie-fa...,Результаты инспекционных мероприятий
693,https://www.mos.ru/pgu/ru/services/link/6365/,Запись в центр госуслуг на кадастровый учет и ...


In [14]:
url_gu_1['Код_ГУ']='-'

In [15]:
# функция для парсинга кода услуги

requests.packages.urllib3.disable_warnings()

def function_2 (j,sem):
    url = url_gu_1.iloc[j]['Ссылка']
    if str(requests.get(url, verify=False, timeout=3)) == '<Response [200]>':
        browser = webdriver.Chrome()
        browser.get(url)
        generated_html = browser.page_source
        browser.quit()
        generated_html
        soup = BeautifulSoup(generated_html, 'html.parser')
        product_names = soup.find_all(class_='css-fe9upj-Button')
        if 'css-fe9upj-Button' in str(soup):
            link_dop = str(product_names).split('href="')[1].split('"')[0].split('/')[-1]
            url_gu_1.loc[j, 'Код_ГУ'] = str(link_dop)
        else:
            url_gu_1.loc[j, 'Код_ГУ'] = '-'
    sem.release()

In [ ]:
# function_2 выполняется в 8 потоков

def main():
    sem = threading.Semaphore(8)
    for j in tqdm(url_gu_1.index):
        sem.acquire()
        threading.Thread(target=function_2, args=(j, sem)).start()

if __name__ ==  "__main__":
    main()

In [17]:
url_gu_1

,Ссылка,Наименование_в_каталоге,Код_ГУ
0,https://www.mos.ru/pgu2/landing/340585/,Посмотреть информацию о посещении и питании в ...,
1,https://www.mos.ru/pgu2/landing/target/7700000...,Запись (перевод) в детский сад,77060101
2,https://www.mos.ru/pgu2/landing/target/7700000...,Компенсация платы за детский сад,-
3,https://www.mos.ru/pgu2/landing/target/7700000...,"Запись в кружки, спортивные секции, дома творч...",077060701
4,https://www.mos.ru/mospaynew/newportal/,Оплата кружков и услуг образования,-
...,...,...,...
690,https://www.mos.ru/pgu2/landing/target/7700000...,Согласие на субаренду либо переуступку прав ар...,-
691,https://www.mos.ru/pgu2/landing/target/7700000...,Выкуп арендуемой недвижимости у города субъект...,-
692,https://www.mos.ru/ggi/function/opredelenie-fa...,Результаты инспекционных мероприятий,-
693,https://www.mos.ru/pgu/ru/services/link/6365/,Запись в центр госуслуг на кадастровый учет и ...,-


In [18]:
url_gu_1.to_excel('Услуги_с_кодом.xlsx')